<a href="https://colab.research.google.com/github/Juliaocmc/GuiIA/blob/main/GuiIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXECUTAR

## Instalações, Importações e configuração da API_KEY


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
import time
from IPython.display import Markdown

Adicione sua API_KEY no painal lateral a esquerda "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
genai.configure(api_key=API_KEY)

## Configurando argumentos

In [ ]:
model = 'gemini-1.0-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MCwidG9wX3AiOjEsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6MjA0OCwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[]

In [ ]:
generation_config

{'temperature': 0,
 'top_p': 1,
 'top_k': 0,
 'max_output_tokens': 2048,
 'stop_sequences': []}

In [ ]:
safety_settings

[{'category': 'HARM_CATEGORY_HARASSMENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_HATE_SPEECH',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'},
 {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
  'threshold': 'BLOCK_MEDIUM_AND_ABOVE'}]

## Chamada da API

In [ ]:
def validar_destino(destino):
  gemini = genai.GenerativeModel(model_name=model)

  chat = gemini.start_chat()

  validador_destino = f'''
  Se for uma cidade, estado ou país, retorne o nome correto do local, se não, responda FALSE.

  Exemplos
  Pergunta: Brasil
  Resposta: Brasil

  Pergunta: Pequim
  Resposta: Pequim

  Pergunta: Goiaba
  Resposta: FALSE

  Pergunta: Estonha
  Resposta: Estônia

  Pergunta:  {destino}
  '''
  response = chat.send_message(validador_destino,stream=stream)
  if (response.text != 'FALSE'):
    print(f'\nGuiIA diz: Legal, seu novo destino é {destino} \nDarei um breve resumo sobre {destino}.\n')
    time.sleep(2)
    prompt_guia = f'De um breve resumo sobre {destino}.'
    breve_resumo(prompt_guia)
  else:
    print(f'\nGuiIA diz: O local {destino} é inválido, digite novamente.')
    destino_correto = input('Destino: ')
    validar_destino(destino_correto)


def breve_resumo(prompt_guia):
  gemini = genai.GenerativeModel(model_name=model)

  resumo = gemini.start_chat()

  response = resumo.send_message(prompt_guia,stream=stream)

  display(Markdown(response.text))

def guia_de_viagem(destino):
  opcoes = f'''\n
  (1) Qual a língua falada em {destino}.
  (2) Quais as principais culinária de {destino}.
  (3) Quais os principais pontos turísticos de {destino}.
  (4) Saber sobre o fuso-horário de {destino}.
  (5) Sobre a temperatura no mês de viagem até {destino}.
  (6) Qual a moeda de {destino}, e suas cotações.
  (7) Sabe sobre outro local.
  (8) Sair.
  '''
  time.sleep(2)
  print(opcoes)
  return input('Qual sua opção: ')

def opcao_1(destino):
  gemini = genai.GenerativeModel(model_name=model)
  validador_opcao_1 = f'''
    Preciso saber qual a língua que se fala em um certo local.
    Se este local a principal língua for a portuguesa, a resposta deve ser
    "Não se preocupe, em .... a língua principal é português. Você vai se dar bem."
    Se não for português, a resposta deve ser o nome da língua principal da cidade.

    Exemplos
    Local: Itália
    Resposta: A língua principal na Itália é o italiano

    Local: Portugal
    Resposta: Não se preocupe, em Portugal a língua principal é português. Você vai se dar bem.

    Local: Japão
    Resposta: A língua principal no Japão na japonês

    Local: {destino}
  '''


  chat_1 = gemini.start_chat()
  response = chat_1.send_message(validador_opcao_1,stream=stream)
  display(Markdown(response.text))
  fala_por_la(destino, response.text)

def fala_por_la(destino, idioma):
  print(f'\nGuiIA diz: Vamos ver alguma curiosidades sobre como falam em {destino}.')
  time.sleep(2)
  gemini = genai.GenerativeModel(model_name=model)
  if "português" in idioma.lower():
    validador_opcao_6_1 = f'''
      Me fale de algumas gírias educadas de {destino}.

      Exemplos:

      "Arretado", gírias nordestina que equivale a bacana, bonito, excelente, legal etc..
      "Mas bá", gírias gaúcha para questionamento ou indagação
    '''
    chat_6_1 = gemini.start_chat()
    response = chat_6_1.send_message(validador_opcao_6_1,stream=stream)
    display(Markdown(response.text))
  else:
    validador_opcao_6_2 = f'''
      Como posso realizar as saudações na língua de {destino},
      "Olá, como vai?"
      "Estou bem."
      "Por favor!"
      "Obrigado!"
      "Boa tarde."
      "Boa noite."
      "Bom dia."


      Exemplos para a Inglês:

      Português: "Olá, como vai?"
      Inglês: "Hello how are you?"

      Português: "Estou bem."
      Inglês: "I am fine."

      Português: "Por favor!"
      Inglês: "Please!"

      Português: "Obrigado!"
      Inglês: "Thanks!"

      Português: "Boa tarde."
      Inglês: "Good afternoon."

      Português: "Boa noite."
      Inglês: "Goodnight."

      Português: "Bom dia."
      Inglês: "Good morning."

    '''
    chat_6_2 = gemini.start_chat()
    response = chat_6_2.send_message(validador_opcao_6_2,stream=stream)
    display(Markdown(response.text))

def opcao_2(destino):
  gemini = genai.GenerativeModel(model_name=model)
  validador_opcao_2 = f'''
    Cite os principais pratos da culinária de {destino}
  '''


  chat_2 = gemini.start_chat()
  response = chat_2.send_message(validador_opcao_2,stream=stream)
  display(Markdown(response.text))

def opcao_3(destino):
  gemini = genai.GenerativeModel(model_name=model)
  validador_opcao_3 = f'''
    Cite os principais pontos turísticos de {destino}
  '''


  chat_3 = gemini.start_chat()
  response = chat_3.send_message(validador_opcao_3,stream=stream)
  display(Markdown(response.text))

def opcao_4(destino):
  gemini = genai.GenerativeModel(model_name=model)
  validador_opcao_4 = f'''
    Qual o horário local em {destino}, e quantas horas de diferença para o horário de brasilia se houver diferenças?
  '''


  chat_4 = gemini.start_chat()
  response = chat_4.send_message(validador_opcao_4,stream=stream)
  display(Markdown(response.text))

def opcao_5(destino):
  gemini = genai.GenerativeModel(model_name=model)
  mes = input(f'\nGuiIA diz: Em qual mês você fará a sua viagem para {destino}?\nResposta: ')

  lista_de_mes = ["1","2","3","4","5","6","7","8","9","10","11","12",
                    "Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho",
                   "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
  if any(elemento.lower() == mes.lower() for elemento in lista_de_mes):
    print(f'\nGuiIA diz: deixa eu pesquisar sobre o clima em {destino} no mês {mes}')
    validador_opcao_5 = f'''
      Me explique um pouco sobre o clima em {destino} no mês de {mes}.
    '''
    chat_5 = gemini.start_chat()
    response = chat_5.send_message(validador_opcao_5,stream=stream)
    display(Markdown(response.text))
  else:
    print(f'\n GuiIA diz: O mês {mes} é inválido. Selecione um mês numérico ou por extenso corretamente.')
    opcao_5(destino)

def opcao_6(destino):
  gemini = genai.GenerativeModel(model_name=model)
  validador_opcao_6 = f'''
      Qual a moeda utilizada em {destino} e quais as cotações para Real, Dólar Americano e Euro.
    '''
  chat_6 = gemini.start_chat()
  response = chat_6.send_message(validador_opcao_6,stream=stream)
  display(Markdown(response.text))


def opcoes(escolha, destino):
  if escolha == "1":
    print(f'\nGuiIA diz: Legal que você quer saber mais sobre o idioma de {destino}. Um momento enquanto eu processo a informação.')
    time.sleep(2)
    opcao_1(destino)


    print(f'\nGuiIA diz: Pressione ENTER para continuar.')
    input()
    print('\n_________________________________________________________________\n')
    print(f'''\nGuiIA diz:E aí, agora que já sabe algo mais sobre {destino}, O que mais gostaria de saber?''')

    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)

  elif escolha == "2":
    print(f'\nGuiIA diz: Boa, vamos saber mais sobre os principais pratos de  {destino}. \nProcessando.')
    time.sleep(2)
    opcao_2(destino)

    print(f'\nGuiIA diz: Pressione ENTER para continuar.')
    input()
    print('\n_________________________________________________________________\n')
    print(f'''\nGuiIA diz: O que mais gostaria de saber sobre {destino}?''')

    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)
  elif escolha == "3":
    print(f'\nGuiIA diz: Vamos conhecer mais sobre os pontos turísticos de {destino}. \nSó um momentinho.')
    time.sleep(2)
    opcao_3(destino)

    print(f'Pressione ENTER para continuar.')
    input()
    print('\n_________________________________________________________________\n')
    print(f'''\nGuiIA diz: O que mais gostaria de saber sobre {destino}?''')

    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)
  elif escolha == "4":
    print(f'\nGuiIA diz: Deixa eu verificar mais sobre o horário de {destino}. \nAguarde só um pouco.')
    time.sleep(2)
    opcao_4(destino)

    print(f'Pressione ENTER para continuar.')
    input()
    print('\n_________________________________________________________________\n')
    print(f'''\nGuiIA diz: O que mais gostaria de saber sobre {destino}?''')

    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)
  elif escolha == "5":
    print(f'\nGuiIA diz: Vamos ver mais sobre o clima em {destino}. Deixa eu processar aqui, só um momento.')
    time.sleep(2)
    opcao_5(destino)

    print(f'Pressione ENTER para continuar.')
    input()
    print('\n_________________________________________________________________\n')
    print(f'''\nGuiIA diz: O que mais gostaria de saber sobre {destino}?''')

    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)
  elif escolha == "6":
    print(f'\nGuiIA diz: Vamos saber sobre a moeda utilizada em {destino}. Deixa eu pesquisar aqui.')
    time.sleep(2)
    opcao_6(destino)

    print(f'Pressione ENTER para continuar.')
    input()
    print('\n_________________________________________________________________\n')
    print(f'''\nGuiIA diz: O que mais gostaria de saber sobre {destino}?''')

    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)
  elif escolha == "7":
    print('\n_________________________________________________________________\n')
    destino = input("\nGuiIA diz: Bora ver mais sobre seu outro destino? \nDestino: ")
    validar_destino(destino)
    print(f'Pressione ENTER para continuar.')
    input()
    print(f'\nO que gostaria de saber sobre {destino}?')
    escolha = guia_de_viagem(destino)
    opcoes(escolha, destino)
  elif escolha == "8":
    print(f'\nGuiIA diz: Muito obrigado. Qualquer dúvida, acesse novamente o GuiIA.')
  else:
    print(f'{escolha} é uma opção inválida, tente novamente.')
    nova_escolha = guia_de_viagem(destino)
    opcoes(nova_escolha, destino)

In [ ]:
def guiia():
  destino = input("\nGuiIA diz: Olá, sou a Inteligência Artificial de apoia a viagem, meu nome é GuiIA.\nQual o seu próximo destino? \nDestino: ")


  validar_destino(destino)
  print(f'Pressione ENTER para continuar.')
  input()
  print(f'''\nGuiIA diz: Quero facilitar sua estadia em {destino}.
    Como posso te ajudar?''')

  escolha = guia_de_viagem(destino)
  opcoes(escolha, destino)

# **GuiIA**

In [ ]:
guiia()